In [1]:
import os
import yaml
import numpy as np
import pandas as pd

In [2]:
datasets = [
    'roman-empire', 'amazon-ratings', 'minesweeper', 'tolokers', 'questions'
]

roc_auc_datasets = ['minesweeper', 'tolokers', 'questions']

models = ['ResNet', 'ResNet+SGC', 'ResNet+adj', 'GCN', 'SAGE', 'GAT', 'GAT-sep', 'GT', 'GT-sep']

In [3]:
df = pd.DataFrame(columns=datasets)

In [4]:
for model in models:
    model_name = model.replace('-', '_').replace('+', '_')
    results = []
    for dataset in datasets:
        val_metrics = []
        metric_name = 'ROC AUC' if dataset in roc_auc_datasets else 'accuracy'
        for num_layers in range(1, 6):
            filename = f'../experiments/{dataset}/{model_name}_l{num_layers}_01/metrics.yaml'
            if os.path.exists(filename):
                with open(filename) as file:
                    metrics = yaml.safe_load(file)

                val_metric_mean = metrics[f'val {metric_name} mean']
                val_metrics.append(val_metric_mean)
        
        if not val_metrics:
            string = ''
        else:
            best_num_layers = np.argmax(val_metrics) + 1
            filename = f'../experiments/{dataset}/{model_name}_l{best_num_layers}_01/metrics.yaml'
            with open(filename) as file:
                metrics = yaml.safe_load(file)

            test_metric_mean = metrics[f'test {metric_name} mean']
            test_metric_std = metrics[f'test {metric_name} std']

            string = f'{test_metric_mean * 100:.2f} \u00B1 {test_metric_std * 100:.2f}'
        
        results.append(string)
    
    df.loc[model] = results

In [5]:
df

,roman-empire,amazon-ratings,minesweeper,tolokers,questions
ResNet,65.88 ± 0.38,45.90 ± 0.52,50.89 ± 1.39,72.95 ± 1.06,70.34 ± 0.76
ResNet+SGC,73.90 ± 0.51,50.66 ± 0.48,70.88 ± 0.90,80.70 ± 0.97,75.81 ± 0.96
ResNet+adj,52.25 ± 0.40,51.83 ± 0.57,50.42 ± 0.83,78.78 ± 1.11,75.77 ± 1.24
GCN,73.69 ± 0.74,48.70 ± 0.63,89.75 ± 0.52,83.64 ± 0.67,76.09 ± 1.27
SAGE,85.74 ± 0.67,53.63 ± 0.39,93.51 ± 0.57,82.43 ± 0.44,76.44 ± 0.62
GAT,80.87 ± 0.30,49.09 ± 0.63,92.01 ± 0.68,83.70 ± 0.47,77.43 ± 1.20
GAT-sep,88.75 ± 0.41,52.70 ± 0.62,93.91 ± 0.35,83.78 ± 0.43,76.79 ± 0.71
GT,86.51 ± 0.73,51.17 ± 0.66,91.85 ± 0.76,83.23 ± 0.64,77.95 ± 0.68
GT-sep,87.32 ± 0.39,52.18 ± 0.80,92.29 ± 0.47,82.52 ± 0.92,78.05 ± 0.93
